In [1]:
!pip install openai

In [2]:
import sys
from pathlib import Path
from datetime import date

CWD = Path().resolve()
PROJECT_ROOT = CWD.parents[0]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.data.supabase_client import (
    fetch_vehicle_by_id,
    upsert_vehicle_timeseries_point,
)
from src.inference.valuation_service import ValuatorEngine

vehicle_id = "f60fab60-3d2d-4fdd-b29b-240b685b3cc3"
vehicle = fetch_vehicle_by_id(vehicle_id)
vehicle

# 2) Run valuation (LLM-based for now)
engine = ValuatorEngine()
valuation = engine.evaluate(vehicle)

print("Estimated value (USD):", valuation.price_usd)
print("Comment:\n", valuation.comment)

# 3) Upsert into vehicle_timeseries for TODAY
today = date.today()
mileage = vehicle.get("mileage") if vehicle else None

result = upsert_vehicle_timeseries_point(
    vehicle_id=vehicle_id,
    value=valuation.price_usd,
    mileage=mileage,
    at_date=today,
)

print("Timeseries upsert result:", result)

Estimated value (USD): 8500.0
Comment:
 The valuation is based on the 2012 Yamaha YZF-R1 with 26,000 miles. The trim is unknown, which could affect the value, but the mileage is reasonable for the age. The overall condition is assumed to be good based on the paddock score of 76. No aftermarket modifications or selected options were provided, which could also influence the price positively or negatively.
Timeseries upsert result: {'action': 'inserted', 'row_id': 'f60fab60-3d2d-4fdd-b29b-240b685b3cc3', 'response': [{'vehicle_id': 'f60fab60-3d2d-4fdd-b29b-240b685b3cc3', 'date': '2025-11-29T00:00:00+00:00', 'value': 8500, 'mileage': 26000, 'time_point_id': '6dfaf11a-73e8-4d15-b323-8b940bd156be'}]}
